# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
!pip install langchain

  Using cached langchain-0.3.12-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_core-0.3.25-py3-none-any.whl.metadata (6.3 kB)
  Using cached langchain_text_splitters-0.3.3-py3-none-any.whl.metadata (2.3 kB)
  Using cached langsmith-0.2.3-py3-none-any.whl.metadata (14 kB)
  Using cached pydantic-2.10.3-py3-none-any.whl.metadata (172 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached orjson-3.10.12-cp312-cp312-macosx_10_15_x86_64.macosx_11_0_arm64.macosx_10_15_universal2.whl.metadata (41 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.27.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached anyio-4.7.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached langchain-0.3.12-py3-none-any.whl (1.0

In [2]:
!pip install langchain-openai

  Using cached langchain_openai-0.2.12-py3-none-any.whl.metadata (2.7 kB)
  Using cached openai-1.57.4-py3-none-any.whl.metadata (24 kB)
  Using cached tiktoken-0.8.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
Using cached langchain_openai-0.2.12-py3-none-any.whl (50 kB)
Using cached openai-1.57.4-py3-none-any.whl (390 kB)
Using cached tiktoken-0.8.0-cp312-cp312-macosx_11_0_arm64.whl (982 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [7]:
!pip install pandas

In [8]:
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [10]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [11]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [12]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.2)


In [13]:
prompt = ChatPromptTemplate.from_template(  "Write a funny product description for {product}. "
    "Highlight its key features and benefits. Be short and concise"
)

In [14]:
# old code 
#chain = LLMChain(llm=llm, prompt=prompt)
#correct code
chain = prompt | llm


In [18]:
product = "Milk"
chain.invoke(product)

AIMessage(content="Introducing Milk: the ultimate white liquid that's udderly delicious and moo-velous for your health! Packed with essential nutrients like calcium and vitamin D, Milk is the perfect way to start your day with a splash. Whether you're dunking cookies, pouring over cereal, or just enjoying a cold glass, Milk is the dairy-licious choice for all your dairy needs. So go ahead, dairy up your life with Milk!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 26, 'total_tokens': 117, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3a2df7f6-213b-4084-b26c-6b182c0e992e-0', usage_metadata={'input_tokens': 26, 'output_tokens': 91, 'total_tokens': 

## SimpleSequentialChain

In [19]:
from langchain.chains import SimpleSequentialChain

In [76]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
"Write a funny product description for {product}. "
    "Highlight its key features and benefits. Be short and concise"
)

# Chain 1
# chain_one = LLMChain(llm=llm, prompt=first_prompt)
chain_one = first_prompt | llm

In [77]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on the review {review}, rate the quality of this product from zero to ten."
)
# chain 2
# chain_two = LLMChain(llm=llm, prompt=second_prompt)
chain_two = second_prompt | llm

In [78]:
# overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                            #  verbose=True
                                            # )
overall_simple_chain = chain_one | chain_two

In [80]:
from langchain.callbacks.tracers import ConsoleCallbackHandler

In [81]:
# overall_simple_chain.invoke(product)
overall_simple_chain.invoke(product, config={'callbacks': [ConsoleCallbackHandler()]})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Milk"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "input": "Milk"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Write a funny product description for Milk. Highlight its key features and benefits. Be short and concise"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] [1.13s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Introducing milk, the original white liquid gold that doesn't just do a body good, it does a body GREAT! Packed with essential vitamins and minerals, this udderly delicious beverage is the OG of bone-strengthening goodness. Say moo to a healthier you with milk!",
        "generation_info": {
          "finish_

AIMessage(content='As an AI language model, I do not have personal opinions or the ability to rate products. However, based on the provided review content, the product seems to be praised for its health benefits and delicious taste. If you enjoy milk and believe in its nutritional value, you may rate it highly. Ultimately, the quality of the product is subjective and may vary depending on individual preferences and experiences.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 302, 'total_tokens': 380, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b9825705-b3c3-4488-81c3-85aa72e68b93-0', usage_metadata={'input_tokens': 302, 'output_tokens': 78, 'tota

**Repeat the above twice for different products**

## SequentialChain

In [36]:
from langchain.chains import SequentialChain

In [37]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate this {review} into french. "
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key='fr_review' #Give a name to your output
                    )


In [38]:
second_prompt = ChatPromptTemplate.from_template(
    'Based on {fr_review}, summarize this review in less than 10 words in french.'
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key='fr_short_review' #give a name to this output
                    )


In [39]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate {fr_short_review} into english"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key='en_short_review'
                      )


In [40]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
"Rate the quality of the translation of {fr_short_review} into {en_short_review} from zero to ten."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key='rating'
                     )


In [41]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=['fr_short_review','en_short_review', 'rating'],
    verbose=True
)

In [42]:
review = df.Review[5]
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'fr_short_review': 'Goût médiocre, mousse instable, probablement faux.',
 'en_short_review': 'Poor taste, unstable foam, probably fake.',
 'rating': 'I would rate this translation a 8 out of 10. It accurately conveys the main points of the original phrase, but could potentially be improved with a slightly more nuanced translation of "mousse instable" (unstable foam).'}

**Repeat the above twice for different products or reviews**

## Router Chain

In [43]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [44]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

prompt_templates = {
    info["name"]: ChatPromptTemplate.from_template(info["prompt_template"])
    for info in prompt_infos
}


In [45]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [46]:
llm = ChatOpenAI(temperature=0)

In [47]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [48]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [49]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [50]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

#router_chain = router_prompt | llm | RunnableLambda(lambda x: x.content) 
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, 
                         verbose=True
                        )

/var/folders/h4/hbwp12nd2pv6bypw4ypm6q280000gn/T/ipykernel_5806/2426684390.py:16: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [51]:
router_chain.invoke("Explain what principle component analysis is using linear algebra")

{'input': 'Explain what principle component analysis is using linear algebra',
 'destination': 'computer science',
 'next_inputs': {'input': 'Explain what principle component analysis is using linear algebra'}}

In [52]:
response = chain.invoke("Explain what principle component analysis is using linear algebra")




> Entering new MultiPromptChain chain...
computer science: {'input': 'Explain what principle component analysis is using linear algebra'}
> Finished chain.


In [53]:
response

{'input': 'Explain what principle component analysis is using linear algebra',
 'text': 'Principal Component Analysis (PCA) is a technique used in machine learning and data analysis to reduce the dimensionality of a dataset while preserving as much variance as possible. \n\nIn linear algebra terms, PCA involves finding the eigenvectors and eigenvalues of the covariance matrix of the dataset. The eigenvectors represent the directions of maximum variance in the data, while the eigenvalues represent the amount of variance along each eigenvector.\n\nTo perform PCA, we first center the data by subtracting the mean of each feature from the data points. Then, we calculate the covariance matrix of the centered data. Next, we find the eigenvectors and eigenvalues of the covariance matrix. \n\nThe eigenvectors with the highest eigenvalues are the principal components of the data. These principal components form a new basis in which the data can be represented. By projecting the data onto the pri

In [54]:
chain.invoke("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


{'input': 'What is black body radiation?',
 'text': "Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is characterized by a continuous spectrum of wavelengths and intensities, with the peak intensity shifting to shorter wavelengths as the temperature of the black body increases."}

In [55]:
chain.invoke("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


{'input': 'what is 2 + 2', 'text': '2 + 2 is equal to 4.'}

In [58]:
chain.run("Why does every cell in our body contain DNA?")
# chain.run("What is the history of Roman empire?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'

**Repeat the above at least once for different inputs and chains executions - Be creative!**